<a href="https://colab.research.google.com/github/jakeryu96/Codestates-Project/blob/main/n224a_model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 4*

# 📝 Assignment
---

# 모델선택(Model Selection)

### 1) 캐글 대회를 이어서 진행합니다. RandomizedSearchCV 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.

- [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)를 사용하세요.
- 분류문제에서 맞는 [scoring parameter](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values) metric을 사용하세요.
- [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) 사용을 권합니다.
- RandomizedSearchCV 를 사용해서 하이퍼파라미터 튜닝을 진행하고 최고 성능을 보이는 모델로 예측을 진행한 후 캐글에 제출합니다.
- **(Urclass Quiz) 캐글 Leaderboard에서 개선된 본인 Score를 과제 제출폼에 제출하세요.**

In [1]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.4 MB/s 


In [2]:
import pandas as pd
import numpy as np

y = 'vacc_h1n1_f'

train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[y], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [3]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, train_size=.8, stratify=train[y], random_state=1)

In [4]:
def engineer(df):    
    dels = [col for col in df.columns if ('employ' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [5]:
X = train.drop(columns=[y]).columns

X_train = train[X]
y_train = train[y]
X_val = val[X]
y_val = val[y]
X_test = test[X]

In [6]:
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

np.seterr(divide='ignore', invalid='ignore')

pipe = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=1,
                           criterion='entropy'))

# 튜닝할 하이퍼파라미터의 범위를 지정해 주는 부분
dists = {
    'simpleimputer__strategy': ['most_frequent', 'median'], 
    'randomforestclassifier__n_estimators': randint(500, 600),
    'randomforestclassifier__max_depth': [5, 10, 20],
    'randomforestclassifier__max_features': [5, 10, 15, 20, None],
    'randomforestclassifier__min_samples_leaf': [10, 50, 100, 200, None],
    'randomforestclassifier__class_weight': ['balanced', 'balanced_subsample', None],
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions=dists, 
    n_iter=50, 
    cv=3,
    scoring='f1',
    verbose=1,
    n_jobs=-1
)

model = clf.fit(X_train, y_train);

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
24 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packa

In [7]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('f1: ', clf.best_score_)

최적 하이퍼파라미터:  {'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 20, 'randomforestclassifier__min_samples_leaf': 50, 'randomforestclassifier__n_estimators': 529, 'simpleimputer__strategy': 'median'}
balanced_accuracy:  0.7444310897879324


In [8]:
model = clf.best_estimator_

In [9]:
def make_submission(test, model):
    ret = pd.Series(model.predict(test), name= 'vacc_h1n1_f')
    ret.index.name = 'id'
    ret.to_csv('mysubmission.csv', index=True)

make_submission(test, model)

from google.colab import files

files.download('mysubmission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🔥 도전과제(Github - Discussion)


### 2) [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 를 사용하여 하이퍼파라미터 튜닝을 진행합니다.
- 모델 성능을 높이기 위해 가능한 시도를 다 해보세요.
- 모델 성능 개선에 가장 큰 영향을 준 특성공학이나 하이퍼파라미터 튜닝에 대해서 왜 성능 개선에 큰 영향을 주었는지 설명해 보시고 서로의 결과에 대해 공유하고 토론해 보세요. 



In [10]:
### 이곳에서 과제를 진행해 주세요 ### 